In [ ]:
import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline

In [ ]:
img = plt.imread('MazeS_10x10.png')
img = np.pad(img,((1,1),(1,1),(0,0)),mode='constant',constant_values=0)
for y in xrange(img.shape[0]):
    for x in xrange(img.shape[1]):
        if img[y,x,:].sum() == 0:
            img[y,x,:] = [1,0,0]

maze_size = 10
maze_px = 1
start = (0,0)
end = (5,5)

img = img[:,:,[2,1,0]]
maze_viz = img.copy()
for y in xrange(end[0]*maze_px,(end[0]+1)*(maze_px)):
    for x in xrange(end[1]*maze_px,(end[1]+1)*(maze_px)):
        maze_viz[y,x,:] = 0.0
plt.imshow(maze_viz,interpolation='nearest')

In [ ]:
up = (0.75,0,0)
down = (0.25,0,0.0)
left = (0.0,0.75,0)
right = (0.0,0.25,0)

sol = np.zeros_like(img)
dist = np.zeros_like(img)

visited = np.zeros_like(img)
visited[maze_viz[:,:,2] > 0.5] = 1

queue = []
for y in xrange(img.shape[0]):
    for x in xrange(img.shape[1]):
        if maze_viz[y,x,:].sum() == 0:
            queue.append([y,x,0,0])
            visited[y,x,:] = 1
steps = 0
while len(queue) > 0:
    y,x,c,s = queue[0]
    queue = queue[1:]
    if y+1 < img.shape[0] and visited[y+1,x,0] < 1:
        queue.append([y+1,x,down,s+1])
    if x+1 < img.shape[1] and visited[y,x+1,0] < 1:
        queue.append([y,x+1,right,s+1])
    if y-1 >= 0 and visited[y-1,x,0] < 1:
        queue.append([y-1,x,up,s+1])
    if x-1 >= 0 and visited[y,x-1,0] < 1:
        queue.append([y,x-1,left,s+1])
    sol[y,x,:] = c
    dist[y,x,:] = s
    visited[y,x,:] = 1
plt.imshow(sol,interpolation='nearest')

In [ ]:
target_pos = np.zeros([sol.shape[0],sol.shape[1],3]) #only using 0 and 1
visited =  np.zeros([sol.shape[0],sol.shape[1]])
visited[maze_viz[:,:,2] > 0.5] = 1

print(sol[0,0,:])
def f(y,x):
    #print y,x
    return { # inverse
        up : (False,(y+1,x)),
        down : (False, (y-1,x)),
        left : (False, (y,x+1)),
        right : (False,(y,x-1)),
        (0,0,0) : (True,(y,x)),
    }[tuple(sol[y,x])]

for y in xrange(img.shape[0]):
    for x in xrange(img.shape[1]):
        if visited[y,x] == 0:
            queue = [(y,x)]
            val = sol[y,x]

            while True:
                yn,xn = queue[-1]
                #print f(yn,xn),queue
                end, new_pos = f(yn,xn)
                #print end,new_pos,queue
                if end:
                    break
                if (sol[new_pos[0],new_pos[1],:] != val).any():
                    break
                queue.append(new_pos)
            #print new_pos,queue
            for q in queue:
                target_pos[q[0],q[1],0] = new_pos[0]
                target_pos[q[0],q[1],1] = new_pos[1]
                visited[q[0],q[1]] = 1
vis_target =target_pos[:,:,:]/np.max(target_pos[:,:,:],(0,1))
plt.imshow(vis_target,interpolation='nearest')

In [ ]:
dist /= dist.max()
plt.subplot(2,2,1)
plt.imshow(maze_viz,interpolation='nearest')
plt.xticks([])
plt.title('maze')
plt.yticks([])
plt.subplot(2,2,2)
plt.imshow(dist,interpolation='nearest')
plt.xticks([])
plt.title('distance to goal')
plt.yticks([])
plt.subplot(2,2,3)
plt.imshow(sol,interpolation='nearest')
plt.title('solution (g: l/r, r:u/d)')
plt.xticks([])
plt.yticks([])
plt.subplot(2,2,4)
plt.imshow(vis_target,interpolation='nearest')
plt.title('target location (red:y,green:x)')
plt.xticks([])
plt.yticks([])
plt.tight_layout()